<a href="https://colab.research.google.com/github/silviootero/Proyecto_IA_ForestPrediction/blob/main/06_Models_with_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost
!pip install catboost
!pip install pyarrow
!pip install fastparquet

In [ ]:
!pip install pyarrow

In [ ]:
!pip install fastparquet

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import sklearn
import xgboost
from sklearn import tree
from sklearn.svm import SVC
from sklearn.metrics import roc_curve
from imblearn.pipeline import Pipeline
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import recall_score, f1_score, roc_auc_score
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

In [ ]:
df = pd.read_parquet('https://mlearner.s3.amazonaws.com/data/clean_data.parquet')
df.head()

In [ ]:
df.columns = df.columns.str.lstrip()

In [ ]:
df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower() for name in df.columns]
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
print(df['id'].value_counts())
sns.countplot(x=df['id'])

In [ ]:
etiquetas = df['id']
df = df.drop(['id'], axis = 1)

In [ ]:
etiquetas

In [ ]:
df

In [ ]:
from sklearn.decomposition import PCA
fil, col = df.shape
from matplotlib import pyplot
import numpy as np
ALL_PCA = PCA(col, random_state=None,
                svd_solver='auto', tol=0.0, whiten=False).fit(df)
ALL_Componentes = np.cumsum(ALL_PCA.explained_variance_ratio_)
print(ALL_Componentes)

In [ ]:
pyplot.plot(np.arange(len(ALL_Componentes)), ALL_Componentes)
pyplot.xlabel('Componentes Principales')
pyplot.ylabel('Varianza Acumulada')
pyplot.title('PCA con Varianza Acumulada')
pyplot.xlim(0, len(ALL_Componentes))
pyplot.ylim(0, 1)
pyplot.show()

In [ ]:
N_COMPONENTS = 65

In [ ]:
from sklearn.decomposition import PCA

pca_df = PCA(n_components=N_COMPONENTS)
pca_df.fit(df)
df_pca = pd.DataFrame(pca_df.transform(df))

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(df_pca, etiquetas, test_size=0.1, stratify = etiquetas, random_state = 42)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=False)

for index_train, index_test in skf.split(X_train, y_train):
    X_train_sm, X_val_sm = X_train.iloc[index_train], X_train.iloc[index_test]
    y_train_sm, y_val_sm = y_train.iloc[index_train], y_train.iloc[index_test]

X_train_sm = X_train_sm.values
X_val_sm = X_val_sm.values
y_train_sm = y_train_sm.values
y_val_sm = y_val_sm.values

In [ ]:
accuracy_lst_reg = []
precision_lst_reg = []
recall_lst_reg = []
f1_lst_reg = []
auc_lst_reg = []

log_reg_sm = LogisticRegression()
log_reg_params = {"penalty": ['l2'],
                  'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  'class_weight': ['balanced',None],
                  'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

rand_log_reg = RandomizedSearchCV(LogisticRegression(), log_reg_params, n_iter=4)


for train, val in skf.split(X_train_sm, y_train_sm):
    pipeline_reg = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_log_reg) # SMOTE happens during Cross Validation not before..
    model_reg = pipeline_reg.fit(X_train_sm[train], y_train_sm[train])
    best_est_reg = rand_log_reg.best_estimator_
    prediction_reg = best_est_reg.predict(X_train_sm[val])
    
    accuracy_lst_reg.append(pipeline_reg.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_reg.append(precision_score(y_train_sm[val], prediction_reg))
    recall_lst_reg.append(recall_score(y_train_sm[val], prediction_reg))
    f1_lst_reg.append(f1_score(y_train_sm[val], prediction_reg))
    auc_lst_reg.append(roc_auc_score(y_train_sm[val], prediction_reg))


print('Logistic Regression (SMOTE) results:')
print('')
print("accuracy: {}".format(np.mean(accuracy_lst_reg)))
print("precision: {}".format(np.mean(precision_lst_reg)))
print("recall: {}".format(np.mean(recall_lst_reg)))
print("f1: {}".format(np.mean(f1_lst_reg)))

In [ ]:
rand_log_reg.best_params_

In [ ]:
label = ['Fin.Stable', 'Fin.Unstable']
pred_reg_sm = best_est_reg.predict(X_val_sm)
print(classification_report(y_val_sm, pred_reg_sm, target_names=label))

In [ ]:
accuracy_lst_xgb = []
precision_lst_xgb = []
recall_lst_xgb = []
f1_lst_xgb = []
auc_lst_xgb = []

xgb_sm = xgboost.XGBClassifier(random_state = 42)
xgb_params = {'eta' : [0.1,0.01,0.001], 
              'eval_metric': ['logloss'],
              'max_depth' : [3,6,9],
              'lambda' : [1,1.5,2],      
              'alpha' : [0,0.5,1]}       
        
rand_xgb = RandomizedSearchCV(xgb_sm, xgb_params, n_iter=4)


for train, val in skf.split(X_train_sm, y_train_sm):
    pipeline_xgb = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_xgb) # SMOTE happens during Cross Validation not before..
    model_xgb = pipeline_xgb.fit(X_train_sm, y_train_sm)
    best_est_xgb = rand_xgb.best_estimator_
    prediction_xgb = best_est_xgb.predict(X_train_sm[val])
    
    accuracy_lst_xgb.append(pipeline_xgb.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_xgb.append(precision_score(y_train_sm[val], prediction_xgb))
    recall_lst_xgb.append(recall_score(y_train_sm[val], prediction_xgb))
    f1_lst_xgb.append(f1_score(y_train_sm[val], prediction_xgb))
    auc_lst_xgb.append(roc_auc_score(y_train_sm[val], prediction_xgb))
    
print('---' * 45)
print('')
print("accuracy: {}".format(np.mean(accuracy_lst_xgb)))
print("precision: {}".format(np.mean(precision_lst_xgb)))
print("recall: {}".format(np.mean(recall_lst_xgb)))
print("f1: {}".format(np.mean(f1_lst_xgb)))
print('---' * 45)

In [ ]:
rand_xgb.best_params_

In [ ]:
smote_prediction_xgb = best_est_xgb.predict(X_val_sm)
print(classification_report(y_val_sm, smote_prediction_xgb, target_names=label))

In [ ]:
accuracy_lst_rfc = []
precision_lst_rfc = []
recall_lst_rfc = []
f1_lst_rfc = []
auc_lst_rfc = []

rfc_sm = RandomForestClassifier()
rfc_params = {'max_features' : ['auto', 'sqrt', 'log2'],
              'random_state' : [42],
              'class_weight' : ['balanced','balanced_subsample'],
              'criterion' : ['gini', 'entropy'],
              'bootstrap' : [True, False]}
    
    
rand_rfc = RandomizedSearchCV(rfc_sm, rfc_params, n_iter=4)


for train, val in skf.split(X_train_sm, y_train_sm):
    pipeline_rfc = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_rfc)
    model_rfc = pipeline_rfc.fit(X_train_sm, y_train_sm)
    best_est_rfc = rand_rfc.best_estimator_
    prediction_rfc = best_est_rfc.predict(X_train_sm[val])
    
    accuracy_lst_rfc.append(pipeline_rfc.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_rfc.append(precision_score(y_train_sm[val], prediction_rfc))
    recall_lst_rfc.append(recall_score(y_train_sm[val], prediction_rfc))
    f1_lst_rfc.append(f1_score(y_train_sm[val], prediction_rfc))
    auc_lst_rfc.append(roc_auc_score(y_train_sm[val], prediction_rfc))
    
print("accuracy: {}".format(np.mean(accuracy_lst_rfc)))
print("precision: {}".format(np.mean(precision_lst_rfc)))
print("recall: {}".format(np.mean(recall_lst_rfc)))
print("f1: {}".format(np.mean(f1_lst_rfc)))

In [ ]:
rand_rfc.best_params_

In [ ]:
label = ['Fin.Stable', 'Fin.Unstable']
smote_prediction_rfc = best_est_rfc.predict(X_val_sm)
print(classification_report(y_val_sm, smote_prediction_rfc, target_names=label))

In [ ]:
accuracy_lst_gbc = []
precision_lst_gbc = []
recall_lst_gbc = []
f1_lst_gbc = []
auc_lst_gbc = []

gbc_sm = GradientBoostingClassifier()
gbc_params = {'max_features' : ['auto', 'sqrt', 'log2'],
              'random_state' : [42],
              'n_estimators' : [20,50,100,200,300]}
    
    
rand_gbc = RandomizedSearchCV(gbc_sm, gbc_params, n_iter=4)


for train, val in skf.split(X_train_sm, y_train_sm):
    pipeline_gbc = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_gbc)
    model_gbc = pipeline_gbc.fit(X_train_sm, y_train_sm)
    best_est_gbc = rand_gbc.best_estimator_
    prediction_gbc = best_est_gbc.predict(X_train_sm[val])
    
    accuracy_lst_gbc.append(pipeline_gbc.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_gbc.append(precision_score(y_train_sm[val], prediction_gbc))
    recall_lst_gbc.append(recall_score(y_train_sm[val], prediction_gbc))
    f1_lst_gbc.append(f1_score(y_train_sm[val], prediction_gbc))
    auc_lst_gbc.append(roc_auc_score(y_train_sm[val], prediction_gbc))
    
print("accuracy: {}".format(np.mean(accuracy_lst_gbc)))
print("precision: {}".format(np.mean(precision_lst_gbc)))
print("recall: {}".format(np.mean(recall_lst_gbc)))
print("f1: {}".format(np.mean(f1_lst_gbc)))

In [ ]:
rand_gbc.best_params_

In [ ]:
smote_prediction_gbc = best_est_gbc.predict(X_val_sm)
print(classification_report(y_val_sm, smote_prediction_gbc, target_names=label))

In [ ]:
accuracy_lst_cbt = []
precision_lst_cbt = []
recall_lst_cbt = []
f1_lst_cbt = []
auc_lst_cbt = []

cbt_sm = CatBoostClassifier(verbose = 0)

cbt_params = {'eval_metric': ['F1'],
              'iterations': [100,500,1000],
              'learning_rate' : [0.1,0.01,0.001],
              'random_seed' : [42],
              'auto_class_weights' : ['Balanced','SqrtBalanced']
             }
    
    
rand_cbt = RandomizedSearchCV(cbt_sm, cbt_params, n_iter=4)

for train, val in skf.split(X_train_sm, y_train_sm):
    pipeline_cbt = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_cbt) # SMOTE happens during Cross Validation not before..
    model_cbt = pipeline_cbt.fit(X_train_sm, y_train_sm)
    best_est_cbt = rand_cbt.best_estimator_
    prediction_cbt = best_est_cbt.predict(X_train_sm[val])
    
    accuracy_lst_cbt.append(pipeline_cbt.score(X_train_sm[val], y_train_sm[val]))
    precision_lst_cbt.append(precision_score(y_train_sm[val], prediction_cbt))
    recall_lst_cbt.append(recall_score(y_train_sm[val], prediction_cbt))
    f1_lst_cbt.append(f1_score(y_train_sm[val], prediction_cbt))
    auc_lst_cbt.append(roc_auc_score(y_train_sm[val], prediction_cbt))
    

print("accuracy: {}".format(np.mean(accuracy_lst_cbt)))
print("precision: {}".format(np.mean(precision_lst_cbt)))
print("recall: {}".format(np.mean(recall_lst_cbt)))
print("f1: {}".format(np.mean(f1_lst_cbt)))

In [ ]:
rand_cbt.best_params_

In [ ]:
smote_prediction_cbt = best_est_cbt.predict(X_val_sm)
print(classification_report(y_val_sm, smote_prediction_cbt, target_names=label))

In [ ]:
best_est_cbt

In [ ]:
rand_cbt.best_params_

In [ ]:
from sklearn.metrics import roc_curve

log_false_positive_rate, log_true_positive_rate, _ = roc_curve(y_val_sm, pred_reg_sm)
xgb_false_positive_rate, xgb_true_positive_rate, _ = roc_curve(y_val_sm, smote_prediction_xgb)
rfc_false_positive_rate, rfc_true_positive_rate, _ = roc_curve(y_val_sm, smote_prediction_rfc)
gbc_false_positive_rate, gbc_true_positive_rate, _ = roc_curve(y_val_sm, smote_prediction_gbc)
cat_false_positive_rate, cat_true_positive_rate, _ = roc_curve(y_val_sm, smote_prediction_cbt)


def graficar_curvas_ROC(log_false_positive_rate, log_true_positive_rate, rfc_false_positive_rate, rfc_true_positive_rate, xgb_false_positive_rate, xgb_true_positive_rate, cat_false_positive_rate, cat_true_positive_rate, gbc_false_positive_rate, gbc_true_positive_rate):
    plt.figure(figsize=(20,8))
    plt.title('Curvas ROC', fontsize=14)
    plt.plot(log_false_positive_rate, log_true_positive_rate, label='Regresión logística AUC: {:.4f}'.format(roc_auc_score(y_val_sm, pred_reg_sm)))
    plt.plot(xgb_false_positive_rate, xgb_true_positive_rate, label='XGBoost AUC: {:.4f}'.format(roc_auc_score(y_val_sm, smote_prediction_xgb)))
    plt.plot(cat_false_positive_rate, cat_true_positive_rate, label='CatBoost AUC: {:.4f}'.format(roc_auc_score(y_val_sm, smote_prediction_cbt)))
    plt.plot(rfc_false_positive_rate, rfc_true_positive_rate, label='Random Forest AUC: {:.4f}'.format(roc_auc_score(y_val_sm, smote_prediction_rfc)))
    plt.plot(gbc_false_positive_rate, gbc_true_positive_rate, label='Gradient Boosting AUC: {:.4f}'.format(roc_auc_score(y_val_sm, smote_prediction_gbc)))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([-0.01, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.legend()
    
graficar_curvas_ROC(log_false_positive_rate, log_true_positive_rate, rfc_false_positive_rate, rfc_true_positive_rate, xgb_false_positive_rate, xgb_true_positive_rate, cat_false_positive_rate, cat_true_positive_rate, gbc_false_positive_rate, gbc_true_positive_rate)

plt.show()

In [ ]:
confusion_matrix_reg = confusion_matrix(y_val_sm, pred_reg_sm, normalize='true')
confusion_matrix_xgb = confusion_matrix(y_val_sm, smote_prediction_xgb, normalize='true')
confusion_matrix_rfc = confusion_matrix(y_val_sm, smote_prediction_rfc, normalize='true')
confusion_matrix_gbc = confusion_matrix(y_val_sm, smote_prediction_gbc, normalize='true')
confusion_matrix_cbt = confusion_matrix(y_val_sm, smote_prediction_cbt, normalize='true')

heat_cm0 = pd.DataFrame(confusion_matrix_reg, columns=np.unique(y_val_sm), index = np.unique(y_val_sm))
heat_cm0.index.name = 'Actual'
heat_cm0.columns.name = 'Predicted'

heat_cm1 = pd.DataFrame(confusion_matrix_xgb, columns=np.unique(y_val_sm), index = np.unique(y_val_sm))
heat_cm1.index.name = 'Actual'
heat_cm1.columns.name = 'Predicted'

heat_cm2 = pd.DataFrame(confusion_matrix_rfc, columns=np.unique(y_val_sm), index = np.unique(y_val_sm))
heat_cm2.index.name = 'Actual'
heat_cm2.columns.name = 'Predicted'

heat_cm3 = pd.DataFrame(confusion_matrix_gbc, columns=np.unique(y_val_sm), index = np.unique(y_val_sm))
heat_cm3.index.name = 'Actual'
heat_cm3.columns.name = 'Predicted'

heat_cm4 = pd.DataFrame(confusion_matrix_cbt, columns=np.unique(y_val_sm), index = np.unique(y_val_sm))
heat_cm4.index.name = 'Actual'
heat_cm4.columns.name = 'Predicted'

f, ax = plt.subplots(1, 5, figsize=(20,8))
f.subplots_adjust(left=None, bottom=None, right= 2, top=None, wspace=None, hspace= None)

sns.heatmap(heat_cm0, cmap="Blues", annot=True, annot_kws={"size": 16},fmt='g', ax = ax[0])
ax[0].set_title('Regresión logística', fontsize = 20)
sns.heatmap(heat_cm1, cmap="Blues", annot=True, annot_kws={"size": 16},fmt='g', ax = ax[1])
ax[1].set_title('XGBoost', fontsize = 20)
sns.heatmap(heat_cm2, cmap="Blues", annot=True, annot_kws={"size": 16},fmt='g', ax = ax[2])
ax[2].set_title('Random Forest', fontsize = 20)
sns.heatmap(heat_cm3, cmap="Blues", annot=True, annot_kws={"size": 16},fmt='g', ax = ax[3])
ax[3].set_title('Gradient Boosting', fontsize = 20)
sns.heatmap(heat_cm4, cmap="Blues", annot=True, annot_kws={"size": 16},fmt='g', ax = ax[4])
ax[4].set_title('Catboost', fontsize = 20)

plt.show()

In [ ]:
test_pred_gbc = best_est_gbc.predict(X_test)
test_pred_cbt = best_est_cbt.predict(X_test)

In [ ]:
confusion_matrix_gbc = confusion_matrix(y_test, test_pred_gbc, normalize='true')
confusion_matrix_cbt = confusion_matrix(y_test, test_pred_cbt, normalize='true')

heat_cm0 = pd.DataFrame(confusion_matrix_gbc, columns=np.unique(y_test), index = np.unique(y_test))
heat_cm0.index.name = 'Actual'
heat_cm0.columns.name = 'Predicted'

heat_cm1 = pd.DataFrame(confusion_matrix_cbt, columns=np.unique(y_test), index = np.unique(y_test))
heat_cm1.index.name = 'Actual'
heat_cm1.columns.name = 'Predicted'

f, ax = plt.subplots(1, 2, figsize=(15,8))
f.subplots_adjust(left=None, bottom=None, right= 2, top=None, wspace=None, hspace= None)

sns.heatmap(heat_cm0, cmap="Blues", annot=True, annot_kws={"size": 16},fmt='g', ax = ax[0])
ax[0].set_title('Gradient Boosting', fontsize = 20)
sns.heatmap(heat_cm1, cmap="Blues", annot=True, annot_kws={"size": 16},fmt='g', ax = ax[1])
ax[1].set_title('Catboost', fontsize = 20)

plt.show()

In [ ]:
print(classification_report(y_test, test_pred_gbc, target_names=label))

In [ ]:
print(classification_report(y_test, test_pred_cbt, target_names=label))